# Imports

In [1]:
!pip install yfinance --upgrade --no-cache-dir
!pip install finnhub-python
!pip3 install news-please

!pip install torch
!pip install peft
!pip install -U accelerate
!pip install transformers bitsandbytes
!pip install pynvml
!pip install nltk
!pip install paramiko
!pip install scp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 304.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 238.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 393.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 126.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for peewee: filename=peewee-3.17.1-cp310-cp310-linux_x86_64.whl size=273724 sha256=ff50842926f9940b4a56d3ca8300fa1377b85174b20cc10cd683d1e40a841ce3
  Stored in directory: /tmp/pip-ephem-wheel-cache-vo9be486/wheels/d7/35/5c/1374782be033462df5f40174d8d879519d64ed8c25a1977554
Successfully built peewee
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparin

In [2]:
# !find / -type f -exec du -h {} + | sort -rh | head -n 10
!rm -rf /root/.cache/*

In [3]:
import paramiko
from scp import SCPClient
server = 'sshhop.hopto.org'
port = 22
user = 'mum'
password = '1234'

def create_ssh_client():
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client

def upload_files(ssh_client, local_path, remote_path):
    with SCPClient(ssh_client.get_transport()) as scp:
        scp.put(local_path, remote_path)  # Use put for uploading

def append_file(ssh_client, local_path, remote_path, temp_path=''):
    
        # Check if temp_path is empty
    if not temp_path:
        temp_path = remote_path + '.tmp'

    upload_files(ssh_client, local_path, temp_path)

    # Step 2: Append the content of the temporary file to the target file
    temp_path = temp_path.replace("/", "\\")
    remote_path = remote_path.replace("/", "\\")
    command = f'type {temp_path} >> {remote_path} & del {temp_path}'
    stdin, stdout, stderr = ssh_client.exec_command(command)
    exit_status = stdout.channel.recv_exit_status()  # Wait for the command to complete
    
    # Reading the output of the command
    output = stdout.read().decode('utf-8')
    error = stderr.read().decode('utf-8')

    # Check if command was successful
    if exit_status == 0:
        print("Command executed successfully")
    else:
        print(f"Command failed with exit status {exit_status}")

    # Optional: Print the outputs for debugging or logging
    if output:
        print("Output:", output)
        
    if error:
        print("Error:", error)

    return not exit_status

def download_files(ssh_client, remote_path, local_path):
    print("downloading")
    with SCPClient(ssh_client.get_transport()) as scp:
        scp.get(remote_path, local_path)
    print("done downloading")

# Optional: Execute a command or run a script on the remote machine
# stdin, stdout, stderr = ssh_client.exec_command('python /path/to/remote/script.py')
# print(stdout.read().decode())  # Assuming the script has output

def close_client(ssh_client):
    # Close the SSH connection
    ssh_client.close()

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
import heapq

def rank_sentences(text):
    stop_words = set(stopwords.words('english'))
    word_frequencies = defaultdict(int)
    
    # Tokenize words and count frequencies, ignoring stopwords
    for word in word_tokenize(text):
        if word.lower() not in stop_words:
            word_frequencies[word.lower()] += 1
    
    # Ensure word_frequencies is not empty before finding the max frequency
    if word_frequencies:
        max_frequency = max(word_frequencies.values())
    else:
        return []
    
    # Normalize frequencies
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / max_frequency)
    
    sentence_scores = defaultdict(int)
    sentences = sent_tokenize(text)
    
    # Score sentences by summing normalized frequencies of their words
    for sent in sentences:
        for word in word_tokenize(sent.lower()):
            if word in word_frequencies:
                sentence_scores[sent] += word_frequencies[word]
    
    # Get top n sentences based on their scores
    summary_sentences = heapq.nlargest(3, sentence_scores, key=sentence_scores.get)
    return summary_sentences

rank_sentences("test. test test.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['test test.', 'test.']

**FinForcaster App.py**


In [5]:
import os
import re
import time
import json
import random
import finnhub
import torch
# import gradio as gr
import pandas as pd
import yfinance as yf
from pynvml import *
from peft import PeftModel
from collections import defaultdict
from datetime import date, datetime, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

finnhub_client = finnhub.Client(api_key="cmr6gghr01qvmr5q06u0cmr6gghr01qvmr5q06ug")

def print_gpu_utilization():

    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def get_curday():

    return datetime.today().strftime("%Y-%m-%d_%H-%M")


def n_weeks_before(date_string, n):
    # date = datetime.strptime(date_string, "%Y-%m-%d_%H-%M") - timedelta(days=7*n)
    thedate = datetime.strptime(date_string, "%Y-%m-%d_%H-%M") - timedelta(days=n)
    return thedate.strftime("%Y-%m-%d")

def n_days_before(date_string, n):
    thedate = datetime.strptime(date_string, "%Y-%m-%d_%H-%M") - timedelta(days=n)
    return thedate.strftime("%Y-%m-%d")


def get_stock_data(stock_symbol, steps):

    stock_data = yf.download(stock_symbol, steps[0], steps[-1])
    if len(stock_data) == 0:
        raise gr.Error(f"Failed to download stock price data for symbol {stock_symbol} from yfinance!")

#     print(stock_data)

    dates, prices = [], []
    available_dates = stock_data.index.format()

    for date in steps[:-1]:
        for i in range(len(stock_data)):
            if available_dates[i] >= date:
                prices.append(stock_data['Close'][i])
                dates.append(datetime.strptime(available_dates[i], "%Y-%m-%d"))
                break

    dates.append(datetime.strptime(available_dates[-1], "%Y-%m-%d"))
    prices.append(stock_data['Close'][-1])

    return pd.DataFrame({
        "Start Date": dates[:-1], "End Date": dates[1:],
        "Start Price": prices[:-1], "End Price": prices[1:]
    })

from dateutil.parser import parse
from zoneinfo import ZoneInfo
def get_news_local(symbol, data, profile, past_hours):
    file_path = f'{symbol}_news.jsonl'
    def download_news():
        ssh_client = create_ssh_client()
        download_files(ssh_client, f'/C:/Users/mum/Documents/news_aggregation_ipynb/news/{file_path}', f'{file_path}')
        close_client(ssh_client)   
    download_news()

    recent_news = []    
    # Calculate 24 hours ago from now
    twenty_four_hours_ago = datetime.now(ZoneInfo("UTC")) - timedelta(hours=past_hours)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            article = json.loads(line)
                # raise Exception(f"No company news found for symbol {symbol} from local!")
            
            # Convert published date from the provided format
            article_date = parse(article.get("published date"))

            # Check if the article's content is not None and the article is within the last 24 hours
            if article.get('content') and article_date > twenty_four_hours_ago:
                # print(rank_sentences(article['content'])[:2])
                summary_sentences = ' '.join(rank_sentences(article['content'])[:2])
                recent_news.append({
                    "date": article_date.strftime('%Y-%m-%d %H:%M'),
                    "headline": article.get('title'),
                    "summary": summary_sentences,
                    "url": article.get('url'),
                    "source": article.get(article['publisher']['title'])
                })
                
        if len(recent_news) == 0:
            print("NO NEWS TO EVALUATE!")

    news_list = []
    news_list.append(json.dumps(recent_news))
    data['News'] = json.dumps(recent_news)
    
    print("Number of news items: ", len(recent_news)) 
    # print(recent_news)
    print(data['News'])
    return data

def get_news(symbol, data, profile):
    
    news_list = []
    filtered_company_name = ' '.join(profile['name'].split()[:1])
    print(profile)

    for end_date, row in data.iterrows():
        print("asdfghjkl")
        start_date = row['Start Date'].strftime('%Y-%m-%d')
        end_date = row['End Date'].strftime('%Y-%m-%d')
#         print(symbol, ': ', start_date, ' - ', end_date)
        # time.sleep(1) # control qpm
        weekly_news = finnhub_client.company_news(symbol, _from=start_date, to=end_date)
        if len(weekly_news) == 0:
            raise gr.Error(f"No company news found for symbol {symbol} from finnhub!")
        weekly_news = [
            {
                "date": datetime.fromtimestamp(n['datetime']).strftime('%Y-%m-%d %H:%M'),
                "headline": n['headline'],
                "summary": n['summary'],
                "url": n['url'],
                "source": n['source']
            } 
            for n in weekly_news
            if "zacks.com" not in n['summary'].lower() and 
               (symbol.lower() in n['headline'].lower() or filtered_company_name.lower() in n['headline'].lower())
        ]
        weekly_news.sort(key=lambda x: x['date'])
        news_list.append(json.dumps(weekly_news))

    data['News'] = news_list
    print(data['News'])

    # additions:
    total_news_count = sum(len(json.loads(weekly_news)) for weekly_news in news_list)
    print("Number of news items after filtering: ", total_news_count) 

    # To save the news to a file, use:
    print_or_save_news(data['News'], f"{symbol}_news.txt")

    return data



In [16]:
def print_or_save_news(news_list, filename=None):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("")
    for news_items_str in news_list:
        news_items = json.loads(news_items_str)  # Load the JSON string into a list of dictionaries
        for news in news_items:
            news_str = f"Date: {news['date']}\nHeadline: {news['headline']}\nSummary: {news['summary']}\nURL: {news['url']}\nSource: {news['source']}\n\n"
            if filename:
                with open(filename, 'a', encoding='utf-8') as file:
                    file.write(news_str)
            else:
                print(news_str)

def get_company_prompt(symbol, profile):

    # profile = finnhub_client.company_profile2(symbol=symbol)
    if not profile:
        raise gr.Error(f"Failed to find company profile for symbol {symbol} from finnhub!")

    company_template = "[Company Introduction]:\n\n{name} is a leading entity in the {finnhubIndustry} sector. Incorporated and publicly traded since {ipo}, the company has established its reputation as one of the key players in the market. As of today, {name} has a market capitalization of {marketCapitalization:.2f} in {currency}, with {shareOutstanding:.2f} shares outstanding." \
        "\n\n{name} operates primarily in the {country}, trading under the ticker {ticker} on the {exchange}. As a dominant force in the {finnhubIndustry} space, the company continues to innovate and drive progress within the industry."

    formatted_str = company_template.format(**profile)

    return formatted_str


def get_prompt_by_row(symbol, row):

    start_date = row['Start Date'] if isinstance(row['Start Date'], str) else row['Start Date'].strftime('%Y-%m-%d')
    end_date = row['End Date'] if isinstance(row['End Date'], str) else row['End Date'].strftime('%Y-%m-%d')
    term = 'increased' if row['End Price'] > row['Start Price'] else 'decreased'
    head = "From {} to {}, {}'s stock price {} from {:.2f} to {:.2f}. Company news during this period are listed below:\n\n".format(
        start_date, end_date, symbol, term, row['Start Price'], row['End Price'])

    news = json.loads(row["News"])
    news = ["[Headline]: {}\n[Summary]: {}\n".format(
        n['headline'], n['summary']) for n in news if n['date'][:8] <= end_date.replace('-', '') and \
        not n['summary'].startswith("Looking for stock market analysis and research with proves results?")]

    basics = json.loads(row['Basics'])
    if basics:
        basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
    else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."

    return head, news, basics


def sample_news(news, k=5):

    return [news[i] for i in sorted(random.sample(range(len(news)), k))]


def get_current_basics(symbol, curday):

    basic_financials = finnhub_client.company_basic_financials(symbol, 'all')
    if not basic_financials['series']:
        raise gr.Error(f"Failed to find basic financials for symbol {symbol} from finnhub!")

    final_basics, basic_list, basic_dict = [], [], defaultdict(dict)

    for metric, value_list in basic_financials['series']['quarterly'].items():
        for value in value_list:
            basic_dict[value['period']].update({metric: value['v']})

    for k, v in basic_dict.items():
        v.update({'period': k})
        basic_list.append(v)

    basic_list.sort(key=lambda x: x['period'])

    for basic in basic_list[::-1]:
        if basic['period'] <= curday:
            break

    return basic


def get_all_prompts_online(symbol, data, curday, profile, with_basics=True):

    company_prompt = get_company_prompt(symbol, profile)

    prev_rows = []

    for row_idx, row in data.iterrows():
        print(row)
        head, news, _ = get_prompt_by_row(symbol, row)
        prev_rows.append((head, news, None))

    prompt = ""
    for i in range(-len(prev_rows), 0):
        prompt += "\n" + prev_rows[i][0]
        sampled_news = sample_news(
            prev_rows[i][1],
            min(5, len(prev_rows[i][1]))
        )
        if sampled_news:
            prompt += "\n".join(sampled_news)
        else:
            prompt += "No relative news reported."

    period = "{} to {}".format(curday, n_weeks_before(curday, -1))

    if with_basics:
        basics = get_current_basics(symbol, curday)
        basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
    else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."

    info = company_prompt + '\n' + prompt + '\n' + basics
    prompt = info + f"\n\nBased on all the information before {curday}, let's first analyze the positive developments and potential concerns for {symbol}. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. " \
        f"Then make your prediction of the {symbol} stock price movement for next week ({period}). Provide a summary analysis to support your prediction."

    return info, prompt


def construct_prompt(ticker, curday, n_weeks, use_basics, past_hours):

    profile = finnhub_client.company_profile2(symbol=ticker)

    try:
        steps = [n_weeks_before(curday, n) for n in range(n_weeks + 1)][::-1]
    except Exception:
        print(f"Invalid date {curday}!")

    data = get_stock_data(ticker, steps)
    data = get_news_local(ticker, data, profile, past_hours) # changed
    data['Basics'] = [json.dumps({})] * len(data)
    # print(data)

    info, prompt = get_all_prompts_online(ticker, data, curday, profile, use_basics)

    prompt = B_INST + B_SYS + SYSTEM_PROMPT + E_SYS + prompt + E_INST
    # print(prompt)

    return info, prompt

ssh_client = create_ssh_client()
def predict(ticker, date, n_weeks, use_basics, past_hours):

    print_gpu_utilization()

    info, prompt = construct_prompt(ticker, date, n_weeks, use_basics, past_hours)

    # additions:
    # save prompt to a file
    with open("./prompt.txt", 'w', encoding='utf-8') as file:
        file.write(prompt)
        file.write("+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-")
    remote_prompttxt_path = f'C:/Users/Mum/Documents/news_aggregation_ipynb/news/{ticker}_PROMPT_from-{date}_past_{past_hours}hours-offset-{offset}.txt'
    append_file(ssh_client, "./prompt.txt", remote_prompttxt_path)

    inputs = tokenizer(
        prompt, return_tensors='pt', padding=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    print("Inputs loaded onto devices.")

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True, streamer=streamer
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)

    torch.cuda.empty_cache()

    return info, answer, output, prompt

In [7]:
base_model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload/"
)
model = PeftModel.from_pretrained(
    base_model,
    'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora',
    offload_folder="offload/"
)
model = model.eval()

tokenizer = AutoTokenizer.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    )

streamer = TextStreamer(tokenizer)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. " \
    "Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ..."

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
count = 0
ssh_client = create_ssh_client()
ticker = "TSLA"
curdate = "2024-02-27_14-30"#get_curday()
print(curdate)
offset = 1
past_hours = 24

def extract_and_append_prediction_to_file(date, offset, past_hours, ticker):
    # Define the pattern to search for
    pattern = r"(Down by \d+-\d+%|Up by \d+-\d+%)"
    # Find all occurrences of the pattern
    matches = re.findall(pattern, answer)
    # Open the file in append mode and write the matches
    
    file_name = f"{ticker}_from-{date}_past_{past_hours}hours-{offset}.txt"
    with open(file_name, 'a', encoding='utf-8') as file:
        for match in matches:
            file.write(match + '\n')
            print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ prediction")

    remote_path = f'C:/Users/Mum/Documents/news_aggregation_ipynb/news/{ticker}_from-{date}_past_{past_hours}hours-offset-{offset}.txt'
    append_file(ssh_client, file_name, remote_path)
    
def extract_and_append_reasoning_to_file(output, date, offset, past_hours, ticker):
    # Open the file in append mode and write the matches
    file_name = f"{ticker}_reasoning_from-{date}_past_{past_hours}hours-offset={offset}.txt"
    with open(file_name, 'a', encoding='utf-8') as file:
        file.write(output + '\n')
        file.write('-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+\n')
        print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ output")

    remote_path = f'C:/Users/Mum/Documents/news_aggregation_ipynb/news/{ticker}_reasoning_from-{date}_past_{past_hours}hours-{offset}.txt'
    append_file(ssh_client, file_name, remote_path)

to_save = {"date": curdate, "past_hours": past_hours, "offset": offset, "predictions": []}
def save_predictions(date, offset, past_hours, ticker, reasoning, prompt):
    # Define the pattern to search for
    pattern = r"(Down by \d+-\d+%|Up by \d+-\d+%)"
    # Find all occurrences of the pattern
    matches = re.findall(pattern, answer)
    # Open the file in append mode and write the matches

    for match in matches:
            to_save['predictions'].append({"prediction": match, "reasoning": reasoning, "prompt": prompt})
            print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+ prediction")

def save_to_save():
    file_name = f'{ticker}_news_predictions.jsonl'
    remote_path = f'C:/Users/Mum/Documents/news_aggregation_ipynb/news/{ticker}_news_predictions.jsonl'
    with open(file_name, 'w') as f:
        json.dump(to_save, f)
    append_file(ssh_client, file_name, remote_path)

# Run the process 10 times
for _ in range(10):
    text, answer, output, prompt = predict(ticker, curdate, offset, False, past_hours)
    # extract_and_append_prediction_to_file()
    save_predictions(curdate, offset, past_hours, ticker, output, prompt)
    # extract_and_append_reasoning_to_file(output, curdate, offset, past_hours, ticker)
    count+=1
    print("-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+", count)

save_to_save()
close_client(ssh_client)

[*********************100%%**********************]  1 of 1 completed

2024-02-27_14-30
GPU memory occupied: 13641 MB.



/tmp/ipykernel_78/3285205267.py:50: FutureWarning: DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  available_dates = stock_data.index.format()
/tmp/ipykernel_78/3285205267.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prices.append(stock_data['Close'][i])
/tmp/ipykernel_78/3285205267.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prices.append(stock_data['Close'][-1])


downloading


In [ ]:
def read_and_print_file(filename):
    try:
        with open(filename, 'r') as file:
            print(f"Contents of {filename}:")
            for line in file:
                print(line.strip())  # .strip() removes any leading/trailing whitespace, including newline characters
    except FileNotFoundError:
        print(f"File {filename} not found.")

# Example usage
filename = "TSLA_from:2024-01-24_past_days:1.txt"
#           TSLA_from:2023-11-9_past_days:3.txt
read_and_print_file(filename)

In [ ]:
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

print(torch.cuda.memory_summary(device=None, abbreviated=False))